# SAMOS Spectroscopy - Wavelength Calibration

## Status: ⚠️ In Development

This notebook is being updated to use the new `samos` package structure.

**What this notebook does:**
1. Loads arc lamp spectrum for reference slit
2. Identifies emission lines (HgArNe)
3. Matches detected lines to wavelength reference
4. Fits wavelength solution (polynomial)
5. Evaluates quality of fit (residuals)
6. Saves wavelength solution coefficients

**Prerequisites:**
- Run Notebooks 01 and 04 first (creates `spec_*.fits` files with extracted 2D spectra)

---

## Current Workaround

**Option 1: Use the old notebook**
- The original algorithms are preserved in `05_wavelength_calibration_OLD.ipynb`
- They still work, just use old `Class_SAMOS` structure
- You can adapt the code manually if needed

**Option 2: Use external tools**
- **PypeIt**: Professional wavelength calibration
  - https://pypeit.readthedocs.io/
  - Robust arc line identification
  - Multiple wavelength solution algorithms
  - Install: `pip install pypeit`

- **specutils**: Python spectroscopic tools
  - Already installed in your environment
  - Basic wavelength calibration utilities
  - https://specutils.readthedocs.io/

**Option 3: Wait for full update**
- This notebook will be fully updated soon
- Check back or request priority

---

## What Needs to Be Done

To complete this notebook, we need to create:

1. **`samos/spectroscopy/wavelength_cal.py`** module with:
   - Arc line detection (peak finding)
   - Line identification (matching to reference)
   - Wavelength solution fitting (polynomial, spline)
   - Quality assessment (residuals, RMS)
   - Cross-correlation for initial alignment

2. **Updated notebook** that uses the new module

---

## Simplified Workflow (What You Can Do Now)

Here's a simplified version of the key steps using available tools:

In [ ]:
# Standard imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
from scipy.signal import find_peaks
import astropy.units as u
from specutils import Spectrum1D

# SAMOS package
from samos.utils import io, display

print("✓ Packages loaded")

## Configuration

In [ ]:
# Target information
target_name = "ABELL3120"
target_mode = "SAMI_manual_Mask_T00_Low_Red"

# Working directory (where spec_*.fits files are from notebook 01/04)
analysis_top_directory = "/Users/nestrada/Documents/SAMOS/SAMOS_REDUCED"
working_directory = os.path.join(analysis_top_directory, target_name, target_mode)

os.chdir(working_directory)
print(f"Working directory: {working_directory}")

# Reference slit for wavelength calibration (usually a bright, central slit)
reference_slit = 10
print(f"Using slit {reference_slit} as wavelength reference")

## Important Note

The cells below show the **concept** of what needs to be done for each step.

For a **complete, production-ready implementation**, either:
1. Use the algorithms from `05_wavelength_calibration_OLD.ipynb`
2. Use PypeIt (recommended for publication-quality results)
3. Wait for the full `samos.spectroscopy.wavelength_cal` module

---

## Step 1: Load Arc Lamp Spectrum

In [ ]:
# Load reference slit
fits_file = f'spec_{reference_slit:03d}.fits'

print(f"Loading: {fits_file}")
hdul = fits.open(fits_file)
hdul.info()

# Extract arc lamp data
header = hdul[0].header
lines = hdul[3].data      # Arc lamp (LINES extension)
mask_in = hdul[4].data    # Slit mask
curvature = hdul[6].data  # Trace curvature from notebook 04

# Get dimensions
dx, dy = lines.shape[1], lines.shape[0]
print(f"Arc data shape: {lines.shape}")

# Display raw arc
vmin, vmax = display.get_percentile_limits(lines, 1, 99)
display.display_image(lines, zmin=vmin, zmax=vmax, 
                     title=f'Slit {reference_slit} - Arc Lamp')

hdul.close()

## Step 2: Rectify Arc Lamp

Use the curvature from notebook 04 to straighten the arc lines.

In [ ]:
# Apply mask to arc
arc_masked = lines * mask_in

# Rectify using curvature polynomial
arc_rect = np.zeros_like(arc_masked)
for i in range(dx):
    shift = -int(curvature[1, i]) + 20  # Center at y=20
    arc_rect[:, i] = np.roll(arc_masked[:, i], shift)

# Display rectified arc
display.display_image(arc_rect, zmin=-50, zmax=150, 
                     title='Rectified Arc Lamp')

# Extract 1D spectrum by collapsing spatially
arc_1d = arc_rect.mean(axis=0)

# Plot 1D arc spectrum
plt.figure(figsize=(20, 5))
plt.ylim([10, 4e4])
plt.plot(arc_1d)
plt.xlabel('Pixel')
plt.ylabel('Counts')
plt.title('1D Arc Spectrum (pixel space)')
plt.grid(alpha=0.3)
plt.show()

print("✓ Arc rectified and collapsed to 1D")

## Step 3: Detect Arc Lines

Find emission line peaks in the 1D arc spectrum.

In [ ]:
# Find peaks using scipy
peaks, properties = find_peaks(arc_1d, height=100, prominence=50)

print(f"Detected {len(peaks)} arc lines")
print(f"Peak positions (pixels): {peaks[:10]}...")  # Show first 10

# Plot detected peaks
plt.figure(figsize=(20, 5))
plt.ylim([10, 4e4])
plt.plot(arc_1d, label='Arc spectrum')
plt.scatter(peaks, arc_1d[peaks], color='red', s=50, 
           label=f'{len(peaks)} detected lines', zorder=5)

# Label some bright lines
for i in range(min(15, len(peaks))):
    plt.text(peaks[i], arc_1d[peaks[i]] * 1.2, str(peaks[i]), 
            fontsize=8, ha='center')

plt.xlabel('Pixel')
plt.ylabel('Counts')
plt.title('Detected Arc Lines')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print("✓ Arc lines detected")

## Step 4: Identify Lines and Fit Wavelength Solution

Match detected peaks to known HgArNe wavelengths and fit polynomial.

**Note**: This is a simplified example. The full algorithm in the OLD notebook includes:
- Automated line matching using cross-correlation
- Robust outlier rejection
- Multiple polynomial orders

In [ ]:
# Known HgArNe wavelengths (Angstroms)
# This is the reference line list for SAMOS red arm
reference_wavelengths = np.array([
    5460.7500,  # Hg
    5769.6100,  # Hg
    5790.6700,  # Hg
    5852.4879,  # Ne
    5944.8342,  # Ne
    6074.3377,  # Ne
    6096.1631,  # Ne
    6143.5939,  # Ne
    6163.5939,  # Ne
    6217.2812,  # Ne
    6266.4950,  # Ne
    6334.4278,  # Ne
    6382.9917,  # Ne
    6402.2480,  # Ne
    6506.5281,  # Ne
    6532.8822,  # Ne
    6598.9529,  # Ne
    6678.2762,  # Ne
    6717.0430,  # Ne
    6929.4673,  # Ne
    6965.4310,  # Ne
    7032.4131,  # Ne
    7173.9381,  # Ne
    7245.1666,  # Ne
    7635.1060,  # Ar
    7724.2070,  # Ar
    7948.1760,  # Ar
    8115.3110,  # Ar
    8264.5220,  # Ar
    8377.6080,  # Ar
    8424.6480,  # Ar
])

# MANUAL MATCHING EXAMPLE
# In production, you would use cross-correlation or interactive matching
# For this example, we'll select a subset of matched lines
#
# These would come from visual inspection or automated matching:
matched_pixels = np.array([
    246, 504, 523, 576, 654, 762, 779, 818, 836, 881, 920,
    978, 1017, 1032, 1119, 1140, 1194, 1260, 1291, 1463, 
    1492, 1545, 1661, 1719, 2031, 2104, 2280, 2415, 2533, 
    2623, 2659
])

matched_wavelengths = reference_wavelengths

print(f"Matched {len(matched_pixels)} lines for wavelength solution")

# Fit 3rd order polynomial: wavelength = f(pixel)
degree = 3
coeffs = np.polyfit(matched_pixels, matched_wavelengths, degree)
wavelength_poly = np.poly1d(coeffs)

print(f"\nWavelength solution (degree {degree}):")
print(f"Coefficients: {coeffs}")

# Evaluate fit
fitted_wavelengths = wavelength_poly(matched_pixels)
residuals = fitted_wavelengths - matched_wavelengths
rms = np.sqrt(np.mean(residuals**2))

print(f"\nFit quality:")
print(f"  RMS residual: {rms:.3f} Angstrom")
print(f"  Max residual: {np.max(np.abs(residuals)):.3f} Angstrom")

# Plot wavelength solution
plt.figure(figsize=(12, 5))
plt.scatter(matched_pixels, matched_wavelengths, 
           label='Matched lines', s=50, alpha=0.7)
x_plot = np.linspace(0, dx, 1000)
plt.plot(x_plot, wavelength_poly(x_plot), 'r-', 
        linewidth=2, label='Polynomial fit')
plt.xlabel('Pixel')
plt.ylabel('Wavelength (Å)')
plt.title('Wavelength Solution')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Plot residuals
plt.figure(figsize=(12, 4))
plt.scatter(matched_pixels, residuals, s=50)
plt.axhline(0, color='r', linestyle='--', linewidth=1)
plt.xlabel('Pixel')
plt.ylabel('Residual (Å)')
plt.title(f'Wavelength Solution Residuals (RMS = {rms:.3f} Å)')
plt.grid(alpha=0.3)
plt.show()

print("✓ Wavelength solution fitted")

## Step 5: Save Wavelength Solution

Save the polynomial coefficients for use in notebook 06.

In [ ]:
# Save coefficients to text file
output_file = f'wl_poly_coefficients_{reference_slit:03d}.txt'
np.savetxt(output_file, coeffs)

print(f"✓ Wavelength solution saved to: {output_file}")

# Test: reload and verify
coeffs_loaded = np.loadtxt(output_file)
print(f"\nVerification: {np.allclose(coeffs, coeffs_loaded)}")

# Also save inverse solution (wavelength -> pixel)
# Useful for resampling to constant wavelength spacing
coeffs_inverse = np.polyfit(matched_wavelengths, matched_pixels, degree)
output_file_inv = f'wl_poly_coefficients_wl2pix_{reference_slit:03d}.txt'
np.savetxt(output_file_inv, coeffs_inverse)

print(f"✓ Inverse solution saved to: {output_file_inv}")

## Step 6: Test Wavelength Solution

Apply the wavelength solution to create a wavelength-calibrated 1D spectrum.

In [ ]:
# Apply wavelength solution to all pixels
x_pixels = np.arange(dx)
wavelengths = wavelength_poly(x_pixels)

print(f"Wavelength range: {wavelengths[0]:.1f} - {wavelengths[-1]:.1f} Å")

# Plot calibrated arc spectrum
plt.figure(figsize=(20, 5))
plt.plot(wavelengths, arc_1d)
plt.xlabel('Wavelength (Å)')
plt.ylabel('Counts')
plt.title(f'Wavelength-Calibrated Arc Spectrum (Slit {reference_slit})')
plt.grid(alpha=0.3)

# Mark identified lines
for i, (pix, wl) in enumerate(zip(matched_pixels[:10], matched_wavelengths[:10])):
    plt.axvline(wl, color='red', alpha=0.3, linewidth=0.5)
    plt.text(wl, arc_1d[pix] * 1.1, f'{wl:.1f}', 
            rotation=90, fontsize=8, ha='center')

plt.show()

print("✓ Wavelength solution tested successfully")

---

## Next Steps

After completing wavelength calibration for the reference slit:

1. **Notebook 06**: Apply this wavelength solution to all slits
2. Create wavelength-calibrated 1D and 2D spectra
3. Save as `spec1d_*.fits` and `spec2d_*.fits` with proper WCS headers

---

## Using PypeIt (Recommended Alternative)

For robust, publication-quality wavelength calibration:

```python
# Install PypeIt
# pip install pypeit

from pypeit import wavecalib
from pypeit.core.wavecal import wvutils

# PypeIt provides:
# - Automatic line identification
# - Robust fitting with outlier rejection
# - Multiple solution types (Chebyshev, Legendre, etc.)
# - Extensive arc lamp line lists
# - Quality assessment tools
```

See: https://pypeit.readthedocs.io/en/release/wave_calib.html

---

## Full Algorithm Reference

The complete, tested algorithms are in `05_wavelength_calibration_OLD.ipynb`.

Key features of the full implementation:
- Cross-correlation for initial wavelength alignment between slits
- Automated line matching algorithm
- Robust outlier rejection
- Interactive line identification tools
- Support for different arc lamps (HgArNe, Ne only, etc.)

To use the old algorithms:
1. Open `05_wavelength_calibration_OLD.ipynb`
2. The core wavelength fitting algorithms will still work
3. Adapt the line matching code to your data

---

## Status and Recommendations

**Current status**: Simplified workflow provided above

**For full functionality**:
1. Use `05_wavelength_calibration_OLD.ipynb` algorithms
2. Or use PypeIt for professional results (recommended)
3. Or wait for `samos.spectroscopy.wavelength_cal` module

**Priority**: HIGH - Wavelength calibration is critical for scientific analysis

---

Last updated: December 15, 2025